In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
%matplotlib inline

In [ ]:
at = pd.read_csv('A-t.csv')
at.replace(' ', np.nan, inplace=True)
print(at)
at = at.astype(float)
print(at.dtypes)
at_long = pd.melt(at,id_vars = 't', var_name = 'A', value_name = 'A_value')
#at_long['A_value'] = pd.to_numeric(at_long['A_value'], errors='coerce')
at_long

In [ ]:
#吸光度散点图
p1 = ggplot(at_long, aes(x = 't', y = 'A_value', color = 'A')) + \
    geom_point()

p1

In [ ]:
A_20_std = np.array([0.829, 0.436, 0.604])
A_30_std = np.array([0.738, 0.66415, 0.612])
c = np.array([0.02015, 0.02015, 0.02728])
l = 1
E_20 = A_20_std / c / l 
E_30 = A_30_std / c / l 
E_20, E_30

In [ ]:
c0 = 0.02015 / 2
at['x_20'] = (at['A20'] / l - c0 * (E_20[0] + E_20[1])) / (E_20[2] - E_20[1] - E_20[0]) 
at['x_30'] = (at['A30'] / l - c0 * (E_30[0] + E_30[1])) / (E_30[2] - E_30[1] - E_30[0]) 
at

In [ ]:
at['f20'] = 1 / (c0 - at['x_20']) - 1 / c0
at['f30'] = 1 / (c0 - at['x_30']) - 1 / c0
at

In [ ]:
#拟合数据处理
fit = at[['t', 'f20', 'f30']]
fit = pd.melt(fit, id_vars = 't', value_vars = ['f20', 'f30'], value_name = 'f', var_name = 'T')
fit

In [ ]:
p = ggplot(fit, aes(x = 't', y = 'f', color = 'T')) + geom_point(size = 1) + geom_smooth(method = 'lm', size = 0.5)
p

In [ ]:
#30度时第一和第二个数据点由于反应并没有进行，因此偏离，故舍弃
fit.drop([13, 14], inplace = True)
fit.dropna(inplace = True)
print(fit)
ggplot(fit, aes(x = 't', y = 'f', color = 'T')) + geom_point(size = 1) + geom_smooth(method = 'lm', size = 0.5)

In [ ]:
#获得线性回归结果
from sklearn.linear_model import LinearRegression
def lr(t):
    model = LinearRegression()
    x = fit[fit['T'] == t]['t'].to_numpy()
    y = fit[fit['T'] == t]['f'].to_numpy()
    model.fit(x.reshape(-1, 1), y.reshape(-1, 1))
    coefficient = model.coef_[0][0]
    intercept = model.intercept_[0]
    print(f"拟合方程: y = {coefficient:.6f} * X + {intercept:.6f}")

In [ ]:
lr('f20')
lr('f30')